In [ ]:
# No change in resolution but varie the timestep
semseg_transform = SemsegTransform()
img_pil = semseg_transform.load('data/semseg_images_exercise_1')
img_pil = semseg_transform.preprocess(img_pil)



# verfiy the normalization
plt.imshow(denormalize(img, mean=IMAGENET_INCEPTION_MEAN, std=IMAGENET_INCEPTION_STD)[0].cpu().permute(1, 2, 0))

res_list = [224, 256, 288, 320, 352, 384, 416, 448]
timestep_list = [0,1, 10, 20, 50, 100]
fig, axes = plt.subplots(len(res_list), len(timestep_list), figsize=(20, 10))

for i,res in enumerate(res_list):
    img_pil = center_crop(img_pil, (min(img_pil.size), min(img_pil.size))).resize((res,res))
    img = semseg_transform.postprocess(img_pil).unsqueeze(0).to(device)
    tokenized_semseg = toks['tok_semseg'].tokenize(img)
    for j,t in enumerate(timestep_list):
        if j == 0:
            axes[i,j].imshow(denormalize(img, mean=IMAGENET_INCEPTION_STD, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
            axes[i,j].set_title("Original ")
            axes[i,j].axis("off")
        else:
            reconstructed_semseg = toks['tok_semseg'].decode_tokens(tokenized_semseg, image_size=res, timesteps=t)
            axes[i,j].imshow(denormalize(reconstructed_semseg, mean=IMAGENET_INCEPTION_STD, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
            axes[i,j].set_title(f"{res},{t}")
            axes[i,j].axis("off")


In [ ]:
# 224 to Higher resolution and 448 to lower resolution
semseg_transform = SemsegTransform()
img_pil = semseg_transform.load('data/semseg_images_exercise_1')
img_pil = semseg_transform.preprocess(img_pil)



# verfiy the normalization
plt.imshow(denormalize(img, mean=IMAGENET_INCEPTION_MEAN, std=IMAGENET_INCEPTION_STD)[0].cpu().permute(1, 2, 0))

res_list_start = [244,448]
res_list = [0,224, 256, 288, 320, 352, 384, 416, 448]
fig, axes = plt.subplots(len(res_list_start),len(res_list), figsize=(20, 10))

for i,res_start in enumerate(res_list_start):
    img_pil = center_crop(img_pil, (min(img_pil.size), min(img_pil.size))).resize((res,res))
    img = semseg_transform.postprocess(img_pil).unsqueeze(0).to(device)
    tokenized_semseg = toks['tok_semseg'].tokenize(img)
    for j,res in enumerate(res_list):
        if i == 0:
            axes[i,j].imshow(denormalize(img, mean=IMAGENET_INCEPTION_STD, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
            axes[i,j].set_title("Original ")
            axes[i,j].axis("off")
        else:
            reconstructed_semseg = toks['tok_semseg'].decode_tokens(tokenized_semseg, image_size=res, timesteps=20)
            axes[i,j].imshow(denormalize(reconstructed_semseg, mean=IMAGENET_INCEPTION_STD, std=IMAGENET_INCEPTION_STD)[0].permute(1, 2, 0).cpu())
            axes[i,j].set_title(f"{res},{t}")
            axes[i,j].axis("off")

In [ ]:
def decode_tok_semseg(rgb_img, mod_dict, tokenizers, key='tok_semseg', image_size=224, patch_size=16, use_detectron=True, return_logits=False):
    """
    Decodes a sequence of semantic segmentation tokens from a model dictionary into an RGB image.

    Args:
        rgb_img (torch.Tensor): RGB image to overlay the semantic segmentation on. # What is this RGB Image => Like a container where the semseg will be transfered
        mod_dict (dict): Model output dictionary.                                  # What is the model output dictionary?
        tokenizers (dict): Dictionary of tokenizers.
        key (str): Key of the tokenized semantic segmentation modality to decode.
        image_size (int): Size of the image.
        patch_size (int): Size of the patches.
        use_detectron (bool): Uses detectron2's visualization for the semseg output.
    """
    tokens = mod_dict[key]['tensor']
    tokens = tokens.unsqueeze(0) if tokens.ndim == 1 else tokens
    img_tok = rearrange(tokens, "b (nh nw) -> b nh nw", nh=image_size//patch_size, nw=image_size//patch_size)
    rec = tokenizers[get_transform_key(key)].decode_tokens(img_tok).detach().cpu()
    if return_logits:
        return rec
    semsegs = rec.argmax(1)
    B, H, W = semsegs.shape

    if not use_detectron:
        return semsegs if B > 1 else semsegs[0]
    else:
        rgb_imgs = [rgb_img] * B
        imgs = []
        for rgb, semseg in zip(rgb_imgs, semsegs):
            if USE_DETECTRON:
                v = Visualizer(255*rgb, coco_metadata, scale=1.2, instance_mode=ColorMode.IMAGE_BW)
                img = v.draw_sem_seg((semseg-1).cpu()).get_image() / 255.0
            else:
                colormap = plt.get_cmap('viridis')
                img = colormap(semseg.cpu())[..., :3]
            imgs.append(img)
        imgs = np_squeeze(np.stack(imgs), axis=0)
        return imgs

In [ ]:
rgb_transform = RGBTransform(imagenet_default_mean_and_std=False)

# Sample the video => Return Numpy array N x H x W x C where N = step 
video_path = 'data/videos_exercise_1'
sampled_frames = load_sampled_frames(video_path, step=10)

# Transform the array to a tensor ( Same memory as numpy array = tensor not resizable)
sample_tensor = torch.from_numpy(sampled_frames) 

for idx in range(sample_tensor[0]):
    sample = sample_tensor[idx,:,:,:]
    # Print Each frame 
